In [ ]:
from collections import namedtuple
from pyspark.sql.functions import col,split, explode
inputDataset="5d63f39d47e8ef16353f41df" # Adobe Analytics: Demo Environment postValues
outputDataset="5d635cefb62f4016359b001b" # Recommendations Input Dataset

In [ ]:
pd1 = spark.read.format("com.adobe.platform.dataset").option('orgId', "1F6324765762BE0E7F000101@AdobeOrg").load(inputDataset)
pd1.describe()

In [ ]:
#Filtering empty values
item_id = "_experience.analytics.customDimensions.eVars.eVar13"
interactionType = "_experience.analytics.customDimensions.eVars.eVar15"
user_id = "_experience.analytics.customDimensions.eVars.eVar1"
brand_name = "_experience.analytics.customDimensions.eVars.eVar16"
timestamp = "timestamp"

pd1 = pd1.filter(pd1[user_id].isNotNull()).filter(pd1[item_id].isNotNull())
pd1 = pd1.filter(pd1[interactionType].isNotNull()).filter(pd1[interactionType].isNotNull())
pd1 = pd1.filter(pd1[brand_name].isNotNull()).filter(pd1[brand_name].isNotNull())

isLaBoutique = pd1[brand_name] == "La Boutique"

pd1 = pd1[isLaBoutique]

pd1.show(100)

In [ ]:
#Splitting item_id to individual records

pd0 = pd1.withColumn("itemId",explode(split(col(item_id), "\|\|")))
pd0.describe()

In [ ]:
df = pd0.selectExpr("_experience.analytics.customDimensions.eVars.eVar1 as userId",
           "_experience.analytics.customDimensions.eVars.eVar13 as itemId",
           "_experience.analytics.customDimensions.eVars.eVar15 as interactionType",
           "_experience.analytics.customDimensions.eVars.eVar16 as brandName",
           "timestamp as timestamp")

df.show(100)

_deloitteemeanorthpartnersand = namedtuple("_deloitteemeanorthpartnersand", ["userId", "itemId", "interactionType", "brandName", "timestamp"])

output = df.rdd.map(lambda parts: { "_deloitteemeanorthpartnersand" : 
                                   _deloitteemeanorthpartnersand(parts[0], parts[1], parts[2], parts[3], parts[4]),"timestamp":parts[4]} )
df_output = output.toDF()
df_output.printSchema()

In [ ]:
userToken = spark.sparkContext.getConf().get('spark.yarn.appMasterEnv.USER_TOKEN')
serviceToken = spark.sparkContext.getConf().get('spark.yarn.appMasterEnv.SERVICE_TOKEN')
serviceApiKey = spark.sparkContext.getConf().get('spark.yarn.appMasterEnv.SERVICE_API_KEY')
df_output.write.format("com.adobe.platform.dataset")\
.option('orgId', "1F6324765762BE0E7F000101@AdobeOrg")\
.option('userToken', userToken)\
.option('serviceToken', serviceToken)\
.option('serviceApiKey', serviceApiKey)\
.save(outputDataset)